<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/8_None_relevant_cases_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold,train_test_split,cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# from imblearn.under_sampling import RandomUnderSampler

from google.colab import drive


In [ ]:
# התחברות ל-Google Drive
drive.mount('/content/drive')



In [ ]:

folder_id = "בית המשפט - קבצים שנבדקו"
directory_path = f"/content/drive/My Drive/{folder_id}"

if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the folder path.")
else:
    dataframes = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_path = os.path.join(directory_path, file_name)
            try:
                df = pd.read_excel(file_path)
                dataframes.append(df)
            except Exception as e:
                print(f"Failed to read {file_name}: {e}")

    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("All Excel files have been concatenated successfully!")

        output_file = "/content/drive/My Drive/combined_excel.xlsx"  # עדכן את הנתיב לשמירת הקובץ
        combined_df.to_excel(output_file, index=False)
        print(f"Combined file saved at: {output_file}")
    else:
        print("No Excel files found or failed to read.")


In [ ]:
combined_df

In [ ]:
data = combined_df.dropna(subset = ['בקשה לרשות ערעור התקבלה?'])

In [ ]:
data

In [ ]:
data['רע"פ בקשה אחרת או דלמטה מיוחד'].value_counts()


In [ ]:
data[(data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'בקשה אחרת') | (data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'דלמטה מיוחד')].shape[0]

In [ ]:
data["אחר"] = data.apply(
    lambda row: 1 if row["בקשה לרשות ערעור התקבלה?"] == "אחר" or not pd.isna(row['רע"פ בקשה אחרת או דלמטה מיוחד']) else 0, axis=1
)

In [ ]:
data["אחר"].value_counts()

In [ ]:
data.drop_duplicates(subset=["מספר הליך"], inplace=True)

In [ ]:
data["אחר"].value_counts()

# Embedding

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

In [ ]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

In [ ]:

# print("מתחילים לחשב אימבדינגים על גוף המסמך המלא...")
# data["גוף המסמך המלא - embeddings"] = data["גוף המסמך"].apply(lambda text: get_embeddings(text))
# print("אימבדינגים 1 חושבו בהצלחה!")


print("מתחילים לחשב אימבדינגים על גוף המסמך החתוך...")
data["גוף המסמך חתוך - embeddings"] = data["גוף המסמך חתוך"].apply(lambda text: get_embeddings(text))
print("אימבדינגים 2 חושבו בהצלחה!")


In [ ]:
output_file = "/content/drive/My Drive/all_OTHER_cases_with_embeddings.xlsx"
data.to_excel(output_file, index=False)
print(f"Combined file saved at: {output_file}")

# Split and train classifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.under_sampling import RandomUnderSampler
import joblib

# הכנת הנתונים
X = np.array(data["גוף המסמך חתוך - embeddings"].tolist())  # הפיכת העמודה לרשימה
y = data["אחר"].values

# שמירת אינדקסים מקוריים
data = data.reset_index(drop=True)  # איפוס האינדקס של ה-DataFrame
original_indices = data.index.values  # שמירת האינדקסים המקוריים

# הוספת אינדקסים לנתונים
indices = np.arange(len(X))
X_with_indices = np.hstack([X, indices.reshape(-1, 1)])  # הוספת אינדקסים כעמודה אחרונה

# ביצוע undersampling
undersampler = RandomUnderSampler(sampling_strategy=0.9, random_state=42)
X_resampled_with_indices, y_resampled = undersampler.fit_resample(X_with_indices, y)

# הפרדת האינדקסים מסט האימון
X_resampled = X_resampled_with_indices[:, :-1]  # כל העמודות פרט לעמודת האינדקסים
indices_resampled = X_resampled_with_indices[:, -1].astype(int)  # עמודת האינדקסים

# יצירת סט הבדיקה מהאינדקסים שלא נבחרו
indices_test = np.setdiff1d(indices, indices_resampled)
X_test = X[indices_test]
y_test = y[indices_test]

# הגדרת משקלות לסיווג לא מאוזן
class_weights = {0: 1, 1: 1}

# יצירת המודל של Logistic Regression
classifier = LogisticRegression(class_weight=class_weights, random_state=42, max_iter=1000)

# אימון המודל על ה-TRAIN SET
classifier.fit(X_resampled, y_resampled)

# חיזוי על ה-TRAIN SET
y_train_pred = classifier.predict(X_resampled)
y_train_pred_prob = classifier.predict_proba(X_resampled)[:, 1]

# חיזוי על ה-TEST SET
y_test_pred = classifier.predict(X_test)
y_test_pred_prob = classifier.predict_proba(X_test)[:, 1]

# שמירת ההסתברויות והחיזוי בדאטה המקורי עבור כל הנתונים
data["Prediction"] = np.nan
data["Prediction_Probability"] = np.nan

# עדכון סט האימון
data.loc[indices_resampled, "Prediction"] = y_train_pred
data.loc[indices_resampled, "Prediction_Probability"] = y_train_pred_prob

# עדכון סט הבדיקה
data.loc[indices_test, "Prediction"] = y_test_pred
data.loc[indices_test, "Prediction_Probability"] = y_test_pred_prob

# חישוב מטריצות בלבול
conf_matrix_train = confusion_matrix(y_resampled, y_train_pred)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)

# דוח ביצועים מלאים
print("\nTrain Set Results:")
print(f"Confusion Matrix (Train):\n{conf_matrix_train}")
print("\nClassification Report (Train):")
print(classification_report(y_resampled, y_train_pred, target_names=["Class 0", "Class 1"]))

print("\nTest Set Results:")
print(f"Confusion Matrix (Test):\n{conf_matrix_test}")
print("\nClassification Report (Test):")
print(classification_report(y_test, y_test_pred, target_names=["Class 0", "Class 1"], zero_division=1))

# שמירת המודל לקובץ לשימוש עתידי
joblib.dump(classifier, 'other_cases_classifier.pkl')
print("Model saved as 'other_cases_classifier.pkl'")


In [ ]:
def export_model_errors(data, filename="model_errors.xlsx"):
    """
    Exports FP and FN errors to an Excel file with the document body and prediction probabilities.

    Parameters:
    - data (DataFrame): The DataFrame containing model predictions and probabilities.
    - filename (str): The name of the output Excel file.

    Returns:
    - None
    """
    # טבלת False Negatives: הערך בפועל 1 אך סווג כ-0
    fn_table = data[(data["אחר"] == 1) & (data["Prediction"] == 0)][["גוף המסמך", "Prediction_Probability"]]
    fn_table["Error Type"] = "FN"  # סוג הטעות

    # טבלת False Positives: הערך בפועל 0 אך סווג כ-1
    fp_table = data[(data["אחר"] == 0) & (data["Prediction"] == 1)][["גוף המסמך", "Prediction_Probability"]]
    fp_table["Error Type"] = "FP"  # סוג הטעות

    # איחוד הטבלאות
    errors_table = pd.concat([fn_table, fp_table], ignore_index=True)


    output_file = "/content/drive/My Drive/Other_cases_classifiers_errors_only_TP_TN.xlsx"  # עדכן את הנתיב לשמירת הקובץ
    errors_table.to_excel(output_file, index=False)
    print(f"Combined file saved at: {output_file}")

export_model_errors(data, filename="model_errors.xlsx")
